In [1]:
import csv
import json
import sqlite3

import requests
import pandas as pd
import numpy as np

from config import api_key, api_key2, api_key3

In [2]:
conn = sqlite3.connect('bls_data.db')
c = conn.cursor()

In [17]:
c.execute("""SELECT series_id
             FROM series_names 
             WHERE datatype = 'All Employees, In Thousands'""")

In [18]:
series_list = []
for i in c:
    series_list.append(i[0])

In [19]:
def export_to_excel(data, file_name):
    writer = pd.ExcelWriter("%s.xlsx" % (file_name,))
    data.to_excel(writer,"%s" % (file_name,))
    writer.save()
    print('Done')
    
def chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

In [20]:
def query_employee_counts(series,startyear,endyear):
    data_dict = {}
    counter = 0
    headers = {'Content-type': 'application/json'}
    data = json.dumps({"seriesid": series,"startyear":startyear, "endyear":endyear, 'registrationkey':api_key3['key']})
    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
    json_data = json.loads(p.text)
    s = json_data['Results']['series']
    for i in s:
        series_id = i['seriesID']
        series_data = i['data']
        for j in series_data:
            year = j['year']
            period = j['period']
            periodName = j['periodName']
            value = j['value']
            data_dict[counter] = pd.Series([series_id,year,period,periodName,value], 
                index=['series_id','year','period','periodName','value'])
            counter+=1
    return pd.DataFrame(data_dict).T

In [21]:
def get_employee_counts(startyear,endyear):
    em_cnt_list = series_list
    ec_list = []
    for i in em_cnt_list:
        ec_list.append(i.rstrip())
    dfs = []
    new_list = chunks(ec_list, 50)
    for n in new_list:
        dfs.append(query_employee_counts(n,startyear,endyear))
    pd.concat(dfs).to_sql('employee_counts', conn, if_exists='replace')
    conn.commit()
    print('Done loading data.')
    return pd.concat(dfs)

In [ ]:
get_employee_counts(2000, 2019)

In [9]:
def query_msa_data():
    data = {}
    c.execute("""SELECT e.series_id, e.year, e.period, e.periodName, e.value, s.area, s.industry, s.datatype, s.adjustment_method
                 FROM employee_counts e 
                 LEFT JOIN series_names s ON e.series_id = s.series_id""")
    counter = 0
    for i in c:
        series_id = i[0]
        year = i[1]
        period = i[2]
        month = i[3]
        month_year = period+'-'+year
        value = i[4]
        msa = i[5]
        industry = i[6]
        datatype = i[7]
        adjustment_method = i[8]
        data[counter] = pd.Series([series_id, year, period, month, month_year, value, msa, industry, datatype, adjustment_method],
                                 index=['series_id', 'year', 'period', 'month', 'month_year', 'value', 'msa', 'industry', 'datatype', 'adjustment_method'])
        counter+=1
    return pd.DataFrame(data).T

In [10]:
df = query_msa_data()

In [11]:
df

,series_id,year,period,month,month_year,value,msa,industry,datatype,adjustment_method
0,SMS01138200000000001,2019,M04,April,M04-2019,546.6,"Birmingham-Hoover, AL",Total Nonfarm,"All Employees, In Thousands",Seasonally Adjusted
1,SMS01138200000000001,2019,M03,March,M03-2019,546.4,"Birmingham-Hoover, AL",Total Nonfarm,"All Employees, In Thousands",Seasonally Adjusted
2,SMS01138200000000001,2019,M02,February,M02-2019,544.6,"Birmingham-Hoover, AL",Total Nonfarm,"All Employees, In Thousands",Seasonally Adjusted
3,SMS01138200000000001,2019,M01,January,M01-2019,544.3,"Birmingham-Hoover, AL",Total Nonfarm,"All Employees, In Thousands",Seasonally Adjusted
4,SMS01138200000000001,2018,M12,December,M12-2018,543.2,"Birmingham-Hoover, AL",Total Nonfarm,"All Employees, In Thousands",Seasonally Adjusted
5,SMS01138200000000001,2018,M11,November,M11-2018,541.8,"Birmingham-Hoover, AL",Total Nonfarm,"All Employees, In Thousands",Seasonally Adjusted
6,SMS01138200000000001,2018,M10,October,M10-2018,541.5,"Birmingham-Hoover, AL",Total Nonfarm,"All Employees, In Thousands",Seasonally Adjusted
7,SMS01138200000000001,2018,M09,September,M09-2018,538.9,"Birmingham-Hoover, AL",Total Nonfarm,"All Employees, In Thousands",Seasonally Adjusted
8,SMS01138200000000001,2018,M08,August,M08-2018,539.3,"Birmingham-Hoover, AL",Total Nonfarm,"All Employees, In Thousands",Seasonally Adjusted
9,SMS01138200000000001,2018,M07,July,M07-2018,538.3,"Birmingham-Hoover, AL",Total Nonfarm,"All Employees, In Thousands",Seasonally Adjusted


In [12]:
export_to_excel(df, 'Examples2000_2019')

Done


In [13]:
get_employee_counts(1980, 1999)

Done loading data.


,series_id,year,period,periodName,value
0,SMS01138200000000001,1999,M12,December,520.5
1,SMS01138200000000001,1999,M11,November,519.1
2,SMS01138200000000001,1999,M10,October,519.0
3,SMS01138200000000001,1999,M09,September,519.3
4,SMS01138200000000001,1999,M08,August,517.7
5,SMS01138200000000001,1999,M07,July,516.8
6,SMS01138200000000001,1999,M06,June,515.6
7,SMS01138200000000001,1999,M05,May,513.7
8,SMS01138200000000001,1999,M04,April,513.4
9,SMS01138200000000001,1999,M03,March,512.2


In [14]:
df2 = query_msa_data()

In [15]:
export_to_excel(df2, 'Examples1980_1999')

Done
